In [1]:
import requests
import json
from IPython.display import display
from bis2 import gc2
from bis import itis

### GAP Species with no TSN
The following report shows that do not currently have an ITIS TSN assigned. At the time these species were identified for modeling, ITIS may have contained no information, but there now may be information in ITIS that could be used. The link below the listing can be used to retrieve information from the ITIS web service.

This list can be reviewed to determine whether or not it is appropriate to go ahead and include a TSN in the metadata for these GAP species, setting up the information for further examination in the TIR.

In [9]:
r_withoutTSN = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration,worms FROM tir.tir WHERE registration->>'source' = 'GAP Species' AND itis->>'MatchMethod' = 'Not Matched'").json()

print ("Number of GAP species with no ITIS designation: ",len(r_withoutTSN["features"]))
print ("------------")

for feature in r_withoutTSN["features"]:
    _registration = json.loads(feature["properties"]["registration"])
    _gapSpeciesCode = _registration["identifiers"][next(index for (index, d) in enumerate(_registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    _scientificname = _registration["scientificname"]
    print (_scientificname, " - ", _registration["commonname"], "(", _gapSpeciesCode, ")")
    print (itis.getITISSearchURL(_scientificname,False))

    if feature["properties"]["worms"] is not None:
        _worms = json.loads(feature["properties"]["worms"])
        if _worms["MatchMethod"] != "Not Matched":
            display (_worms)

    print ("------------")
    


Number of GAP species with no ITIS designation:  25
------------
Cervus elaphus nannodes roosevelti  -  Roosevelt Elk ( mELK1r )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Cervus\%20elaphus\%20nannodes\%20roosevelti
------------
Sylvilagus palustris heferni  -  Lower Keys Rabbit ( mMARAh )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Sylvilagus\%20palustris\%20heferni
------------
Corynorhinus townsendii ingens  -  Ozark Big-eared Bat ( mTBEBi )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Corynorhinus\%20townsendii\%20ingens
------------
Urocitellus elegans elegans  -  Wyoming Ground Squirrel elegans ( mWYSQe )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Urocitellus\%20elegans\%20elegans
------------
Corynorhinus townsendii australis  -  Townsend's Big-eared Bat australis ( mTBEBa )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Corynorhinus\%20townsendii\%20australis
------------
Sceloporus uniformis  -  Yellow-backed Spiny Lizard ( rYBSL

### Invalid TSNs
The following report shows GAP species where the assigned TSN is currently showing as invalid from ITIS. The information displayed includes a link to the invalid TSN from the ITIS web service, the reason the name is invalid, and a link to the ITIS service for the accepted TSN.

This may reflect a change in the taxonomy since the GAP species was identified and set up for modeling. These records can be reviewed to determine if some action is appropriate in the GAP species metadata.

In [5]:
r_invalidTSN = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration,itis FROM tir.tir WHERE registration->>'source' = 'GAP Species' AND itis->>'usage' <> 'valid'").json()

print ("Number of GAP species with invalid ITIS TSN: ",len(r_invalidTSN["features"]))
print ("------------")

for feature in r_invalidTSN["features"]:
    _registration = json.loads(feature["properties"]["registration"])
    _itis = json.loads(feature["properties"]["itis"])
    _gapSpeciesCode = _registration["identifiers"][next(index for (index, d) in enumerate(_registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    print (_registration["scientificname"], " - ", _registration["commonname"], "(", _gapSpeciesCode, ")")
    print ("Assigned (invalid) TSN: ",itis.getITISSearchURL(_itis["tsn"],False))
    print ("Unaccept Reason: ",_itis["unacceptReason"])
    print ("Accepted TSN: ",itis.getITISSearchURL(_itis["acceptedTSN"][0],False))
    print ("------------")


Number of GAP species with invalid ITIS TSN:  128
------------
Myodes californicus  -  Western Red-backed Vole ( mWRBVx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:180295
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:970610
------------
Wilsonia citrina  -  Hooded Warbler ( bHOWAx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:178972
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:950029
------------
Dendroica palmarum  -  Palm Warbler ( bPAWAx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:178921
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:950043
------------
Spermophilus armatus  -  Uinta Ground Squirrel ( mUGSQx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:180147
Unaccept Reason:  o

### WoRMS Information

In addition to consulting ITIS, the TIR looks for taxa in the World Register of Marine Species. The following report shows GAP species that were found in the WoRMS system. The full data structure captured from WoRMS is shown for evaluation.

In [14]:
r_gapSpeciesInWoRMS = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration,worms FROM tir.tir WHERE registration->>'source' = 'GAP Species' AND worms->>'MatchMethod' <> 'Not Matched'").json()

print ("Number of GAP species with WoRMS information: ",len(r_gapSpeciesInWoRMS["features"]))
print ("------------")

for feature in r_gapSpeciesInWoRMS["features"]:
    _registration = json.loads(feature["properties"]["registration"])
    _worms = json.loads(feature["properties"]["worms"])
    _gapSpeciesCode = _registration["identifiers"][next(index for (index, d) in enumerate(_registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    print (_registration["scientificname"], " - ", _registration["commonname"], "(", _gapSpeciesCode, ")")
    display (_worms)
    print ("------------")


Number of GAP species with WoRMS information:  319
------------
Rattus norvegicus  -  Brown Rat ( mBRRTx )


{'AphiaID': 993634,
 'MatchMethod': 'Exact Match',
 'MatchString': 'Rattus norvegicus',
 'cacheDate': '2017-07-26T15:08:03.419808',
 'class': 'Mammalia',
 'family': 'Muridae',
 'genus': 'Rattus',
 'isBrackish': 0,
 'isExtinct': None,
 'isFreshwater': 0,
 'isMarine': 0,
 'isTerrestrial': 1,
 'kingdom': 'Animalia',
 'lsid': 'urn:lsid:marinespecies.org:taxname:993634',
 'match_type': 'exact',
 'modified': '2017-05-30T15:14:47Z',
 'order': 'Rodentia',
 'phylum': 'Chordata',
 'rank': 'Species',
 'scientificname': 'Rattus norvegicus',
 'status': 'accepted',
 'unacceptreason': None,
 'valid_AphiaID': 993634,
 'valid_name': 'Rattus norvegicus'}

------------
Wilsonia pusilla  -  Wilson's Warbler ( bWIWAx )


{'AphiaID': 422662,
 'MatchMethod': 'Exact Match',
 'MatchString': 'Wilsonia pusilla',
 'cacheDate': '2017-07-26T15:09:17.254067',
 'class': 'Aves',
 'family': 'Parulidae',
 'genus': 'Wilsonia',
 'isBrackish': 0,
 'isExtinct': None,
 'isFreshwater': None,
 'isMarine': 0,
 'isTerrestrial': None,
 'kingdom': 'Animalia',
 'lsid': 'urn:lsid:marinespecies.org:taxname:422662',
 'match_type': 'exact',
 'modified': '2011-04-08T08:18:06Z',
 'order': 'Passeriformes',
 'phylum': 'Chordata',
 'rank': 'Species',
 'scientificname': 'Wilsonia pusilla',
 'status': 'accepted',
 'unacceptreason': None,
 'valid_AphiaID': 422662,
 'valid_name': 'Wilsonia pusilla'}

------------
Dendroica coronata  -  Yellow-rumped Warbler ( bYRWAx )


...

------------
Hirundo rustica  -  Barn Swallow ( bBARSx )


{'AphiaID': 212750,
 'MatchMethod': 'Exact Match',
 'MatchString': 'Hirundo rustica',
 'cacheDate': '2017-07-26T15:06:46.844044',
 'class': 'Aves',
 'family': 'Hirundinidae',
 'genus': 'Hirundo',
 'isBrackish': 0,
 'isExtinct': None,
 'isFreshwater': None,
 'isMarine': 0,
 'isTerrestrial': 1,
 'kingdom': 'Animalia',
 'lsid': 'urn:lsid:marinespecies.org:taxname:212750',
 'match_type': 'exact',
 'modified': '2011-04-08T08:18:06Z',
 'order': 'Passeriformes',
 'phylum': 'Chordata',
 'rank': 'Species',
 'scientificname': 'Hirundo rustica',
 'status': 'accepted',
 'unacceptreason': None,
 'valid_AphiaID': 212750,
 'valid_name': 'Hirundo rustica'}

------------
Arctocephalus townsendi  -  Guadalupe Fur Seal ( mGFSEx )


{'AphiaID': 255007,
 'MatchMethod': 'Exact Match',
 'MatchString': 'Arctocephalus townsendi',
 'cacheDate': '2017-07-26T15:07:08.081917',
 'class': 'Mammalia',
 'family': 'Otariidae',
 'genus': 'Arctocephalus',
 'isBrackish': None,
 'isExtinct': None,
 'isFreshwater': None,
 'isMarine': 1,
 'isTerrestrial': None,
 'kingdom': 'Animalia',
 'lsid': 'urn:lsid:marinespecies.org:taxname:255007',
 'match_type': 'exact',
 'modified': '2010-05-20T12:09:50Z',
 'order': 'Carnivora',
 'phylum': 'Chordata',
 'rank': 'Species',
 'scientificname': 'Arctocephalus townsendi',
 'status': 'unaccepted',
 'unacceptreason': None,
 'valid_AphiaID': 477319,
 'valid_name': 'Arctocephalus philippii townsendi'}

------------


### FWS Listing Status

The TIR looks for information from the US Fish and Wildlife Service Threatened and Endangered Species System (TESS) for registered species. The following report shows listing status for GAP species. Listing status here is stored as a data structure that can contain more than one set of listing details for different populations where a species has specific designations. This information could be further synthesized in some way to produce a more simplified way of filtering GAP species for specific purposes.

In [13]:
r_gapSpeciesTESS = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration,tess FROM tir.tir WHERE registration->>'source' = 'GAP Species'").json()

for feature in r_gapSpeciesTESS["features"]:
    _registration = json.loads(feature["properties"]["registration"])
    _tess = json.loads(feature["properties"]["tess"])
    _gapSpeciesCode = _registration["identifiers"][next(index for (index, d) in enumerate(_registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    print (_registration["scientificname"], " - ", _registration["commonname"], "(", _gapSpeciesCode, ")")
    if _tess["result"]:
        print ("Date Cached: ",_tess["dateCached"])
        display (_tess["listingStatus"])
    else:
        print ("Not listed")
    print ("------------")  


Megascops asio  -  Eastern Screech-owl ( bEASOx )
Not listed
------------
Rattus norvegicus  -  Brown Rat ( mBRRTx )
Not listed
------------
Plethodon dorsalis  -  Northern Zigzag Salamander ( aNZSAx )
Not listed
------------
Wilsonia pusilla  -  Wilson's Warbler ( bWIWAx )
Not listed
------------
Empidonax virescens  -  Acadian Flycatcher ( bACFLx )
Not listed
------------
Peromyscus truei comanche  -  Palo Duro Deermouse ( mPIDEc )
Date Cached:  2017-07-26T15:54:42.935617


[{'POP_ABBREV': 'Wherever found',
  'POP_DESC': 'Wherever found',
  'STATUS': 'Species of Concern'}]

------------
Peromyscus eremicus  -  Cactus Deermouse ( mCCDEx )
Not listed
------------
Phaeognathus hubrichti  -  Red Hills Salamander ( aRHSAx )
Date Cached:  2017-07-26T15:54:58.044765


[{'LISTING_DATE': '1977-01-03',
  'POP_ABBREV': 'Wherever found',
  'POP_DESC': 'Wherever found',
  'STATUS': 'Threatened'}]

------------
Eurycea longicauda  -  Long-tailed Salamander ( aLTASx )
Not listed
------------
Apalone mutica calvata  -  Gulf Coast Smooth Softshell ( rSMSOc )
Not listed
------------
Kinosternon sonoriense  -  Sonoran Mud Turtle ( rSOMTx )
Not listed
------------
Dendroica coronata  -  Yellow-rumped Warbler ( bYRWAx )
Not listed
------------
Tympanuchus cupido  -  Greater Prairie-chicken ( bGRPCx )
Not listed
------------
Cryptotis parva  -  North American Least Shrew ( mNALSx )
Not listed
------------
Lasiurus blossevillii  -  Western Red Bat ( mWRBAx )
Not listed
------------
Synaptomys cooperi  -  Southern Bog Lemming ( mSBLEx )
Not listed
------------
Leucophaeus atricilla  -  Laughing Gull ( bLAGUx )
Not listed
------------
Peromyscus polionotus peninsularis  -  St. Andrews Beach Deermouse ( mOLDEp )
Date Cached:  2017-07-26T15:55:31.475992


...

------------
Plethodon aureolus  -  Tellico Salamander ( aTESAx )
Not listed
------------
Actitis macularius  -  Spotted Sandpiper ( bSPSAx )
Not listed
------------
Pseudacris streckeri  -  Strecker's Chorus Frog ( aSTFRx )
Not listed
------------
Hirundo rustica  -  Barn Swallow ( bBARSx )
Not listed
------------
Drymarchon couperi  -  Eastern Indigo Snake ( rEISNx )
Not listed
------------
Blarina brevicauda  -  Northern Short-tailed Shrew ( mNOSSx )
Not listed
------------
Storeria dekayi  -  Dekay's Brownsnake ( rDEBRx )
Not listed
------------
Desmognathus welteri  -  Black Mountain Salamander ( aBMSAx )
Not listed
------------
Reithrodontomys megalotis  -  Western Harvest Mouse ( mWHMOx )
Not listed
------------
Arctocephalus townsendi  -  Guadalupe Fur Seal ( mGFSEx )
Date Cached:  2017-07-26T15:54:19.819007


[{'LISTING_DATE': '1967-03-11',
  'POP_ABBREV': 'Wherever found',
  'POP_DESC': 'Wherever found',
  'STATUS': 'Threatened'}]

------------
Lithobates heckscheri  -  River Frog ( aRIFRx )
Not listed
------------
Peromyscus polionotus leucocephalus  -  Santa Rosa Beach Deermouse ( mOLDEl )
Date Cached:  2017-07-26T15:54:27.117673


[{'POP_ABBREV': 'Wherever found',
  'POP_DESC': 'Wherever found',
  'STATUS': 'Species of Concern'}]

------------
